In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score
import random

In [2]:
travel_styl_mapping = {
    1: '자연 선호 매우선호',
    2: '자연 선호 중간선호',
    3: '자연 선호 약간선호',
    4: '중립',
    5: '도시 선호 약간선호',
    6: '도시 선호 중간선호',
    7: '도시 선호 매우선호'
}

In [4]:
# 데이터 불러오기 및 전처리
place = pd.read_csv('/content/travel_data.csv')
place = place.drop(['TRAVEL_ID','TRAVELER_ID','TRAVEL_PURPOSE'], axis=1)

place['AGE_GRP'] = place['AGE_GRP'].astype('int16')
place['TRAVEL_STYL_1'] = place['TRAVEL_STYL_1'].astype('int16')

# 시군구 1, 2, 3을 하나의 리스트로 합치기
all_sgg = pd.concat([place['TRAVEL_LIKE_SGG_1'], place['TRAVEL_LIKE_SGG_2'], place['TRAVEL_LIKE_SGG_3']])

# 가중치
sgg_counts = all_sgg.value_counts()
sgg_weights = 1 / sgg_counts
sgg_weights = sgg_weights / sgg_weights.sum()

# 각 행에 대해 가중치 계산 함수 정의
def calculate_weight(row):
    sgg_1_weight = sgg_weights.get(row['TRAVEL_LIKE_SGG_1'], 0)
    sgg_2_weight = sgg_weights.get(row['TRAVEL_LIKE_SGG_2'], 0)
    sgg_3_weight = sgg_weights.get(row['TRAVEL_LIKE_SGG_3'], 0)
    return (sgg_1_weight + sgg_2_weight + sgg_3_weight) / 3

# 각 행에 대해 가중치 계산
place['WEIGHTS'] = place.apply(calculate_weight, axis=1)

# SGG 1,2,3순위 합치기
place['TRAVEL_LIKE_SGG'] = place[['TRAVEL_LIKE_SGG_1', 'TRAVEL_LIKE_SGG_2', 'TRAVEL_LIKE_SGG_3']].apply(lambda x: ','.join(x.dropna()), axis=1)
place = place.drop(columns=['TRAVEL_LIKE_SGG_1', 'TRAVEL_LIKE_SGG_2', 'TRAVEL_LIKE_SGG_3'])
place.head()

,GENDER,AGE_GRP,TRAVEL_STATUS_RESIDENCE,TRAVEL_STYL_1,WEIGHTS,TRAVEL_LIKE_SGG
0,남,40,경기도,5,0.000135,"제주특별자치도 제주시,강원도 속초시,강원도 동해시"
1,남,30,서울특별시,3,0.000107,"부산광역시 해운대구,부산광역시 기장군,제주특별자치도 제주시"
2,여,30,인천광역시,3,0.000409,"강원도 강릉시,서울특별시 마포구,경기도 포천시"
3,여,30,경상남도,5,0.000275,"서울특별시 송파구,서울특별시 마포구,부산광역시 해운대구"
4,여,60,경상남도,3,0.000564,"경상남도 통영시,서울특별시 영등포구,부산광역시 기장군"


In [ ]:
area = pd.read_csv('/content/visit_area_processing.csv')
area = area[['VISIT_AREA_NM', 'ROAD_NM_ADDR']]
area = area.dropna(subset=['ROAD_NM_ADDR', 'VISIT_AREA_NM'])
area = area.drop_duplicates()
area = area.reset_index(drop=True)

In [6]:
# 랜덤 유저 생성
def get_user():
    gender = random.choice(['남', '여'])
    age = random.choice([10,20,30,40,50,60,70])
    resi = random.choice(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시',
                            '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도'])
    styl = random.choice(list(travel_styl_mapping.keys()))
    return  gender, age, resi, styl

In [7]:
# 라벨 인코더와 스케일러 학습
le_sgg = LabelEncoder()
le_gender = LabelEncoder()
le_residence = LabelEncoder()

place['TRAVEL_LIKE_SGG'] = le_sgg.fit_transform(place['TRAVEL_LIKE_SGG'])
place['GENDER'] = le_gender.fit_transform(place['GENDER'])
place['TRAVEL_STATUS_RESIDENCE'] = le_residence.fit_transform(place['TRAVEL_STATUS_RESIDENCE'])

scaler = MinMaxScaler()
place[['AGE_GRP', 'TRAVEL_STYL_1']] = scaler.fit_transform(place[['AGE_GRP', 'TRAVEL_STYL_1']])

# 목표 변수 설정
target = place['TRAVEL_LIKE_SGG']

# 특성 변수 설정 (WEIGHTS 제외)
features = place.drop(columns=['TRAVEL_LIKE_SGG', 'WEIGHTS'])

# Train-Test Split
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(features, target, place['WEIGHTS'], test_size=0.3, random_state=42)

# LightGBM 데이터셋 생성
train_data = lgb.Dataset(X_train, label=y_train, weight=w_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# LightGBM 모델 학습
params = {
    'objective': 'multiclass',
    'num_class': len(le_sgg.classes_),
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.8
}
num_round = 80
model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# 예측 함수
def predicted(gender, age, residence, style):
    user = pd.DataFrame({
        'GENDER': [gender],
        'AGE_GRP': [age],
        'TRAVEL_STATUS_RESIDENCE': [residence],
        'TRAVEL_STYL_1': [style],
    })

    user['GENDER'] = le_gender.transform(user['GENDER'])
    user['TRAVEL_STATUS_RESIDENCE'] = le_residence.transform(user['TRAVEL_STATUS_RESIDENCE'])
    user[['AGE_GRP', 'TRAVEL_STYL_1']] = scaler.transform(user[['AGE_GRP', 'TRAVEL_STYL_1']])

    prediction_prob = model.predict(user, num_iteration=model.best_iteration)
    prediction_class = np.argmax(prediction_prob, axis=1)
    predict_sgg = le_sgg.inverse_transform(prediction_class)[0]

    first_predicted_sgg = predict_sgg.split(',')[0]

    filtered_df = area[area['ROAD_NM_ADDR'].str.contains(first_predicted_sgg)]
    recommended_areas = filtered_df['VISIT_AREA_NM'].head(10).tolist()

    return first_predicted_sgg, recommended_areas

# 사용자 입력 및 예측
for _ in range(10):
    user_gender, user_age, user_residence, user_style = get_user()
    predicted_location, predicted_place = predicted(user_gender, user_age, user_residence, user_style)

    mapped_style = travel_styl_mapping.get(user_style, "Unknown")

    print(f' Gender: {user_gender}, Age: {user_age}, Residence: {user_residence}')
    print(f'Travel Style: {mapped_style}' )
    print(f'Predicted Travel SGG: {predicted_location}')
    print(f'Recommend Places : {predicted_place}')
    print('==========================')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix

# 모델 평가
y_pred_prob = model.predict(X_test, num_iteration=model.best_iteration)
y_pred = np.argmax(y_pred_prob, axis=1)

# 실제 클래스와 예측 클래스를 디코딩
y_test_decoded = le_sgg.inverse_transform(y_test)
y_pred_decoded = le_sgg.inverse_transform(y_pred)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Confusion Matrix:')
print(conf_matrix)

Accuracy: 0.0000
Confusion Matrix:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
